In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(""),".."))

import numpy
import pandas
from IPython.display import display
import custom
from kiwipiepy import Kiwi
from tqdm import tqdm


In [2]:
kiwi = Kiwi()

print(kiwi.tokenize("안녕하세요"))
print(kiwi.tokenize("진짜 조낸 재밌다 굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿"))

[Token(form='안녕', tag='NNG', start=0, len=2), Token(form='하', tag='XSA', start=2, len=1), Token(form='세요', tag='EF', start=3, len=2)]
[Token(form='진짜', tag='MAG', start=0, len=2), Token(form='조', tag='NNP', start=3, len=1), Token(form='내', tag='NNB', start=4, len=1), Token(form='ᆫ', tag='JX', start=4, len=1), Token(form='재밌', tag='VA', start=6, len=2), Token(form='다', tag='EF', start=8, len=1), Token(form='굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿', tag='NNG', start=10, len=22)]


In [3]:
original_df = pandas.read_csv("data/ratings_test.txt", sep = "\t", encoding = "UTF8")

display(original_df)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [4]:
df = original_df[['document','label']]

for i in tqdm(range(len(df))) :
    text = custom.text_preprocess_kor(str(df.iloc[i,0]), end_mark = True)
    if text == " " :
        df.iloc[i,0] = text
        continue
    try :
        temp = kiwi.tokenize(text)
        temp_list = []
        for w in temp :
            temp_list.append(w.form)
        df.iloc[i,0] = " ".join(temp_list)
    except Exception as e :
        df.iloc[i,0] = " "
        print(str(e), f"문장 : {text}")
    
word_len = []
for i in range(len(df)) :
    word_len.append(len(str(df.iloc[i,0]).split()))
df["word_len"] = word_len

df = df.drop(df[df["word_len"] == 0].index, axis = 0)

display(df)


100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [02:04<00:00, 400.04it/s]


,document,label,word_len
0,굳,1,1
2,뭐 이 야 이 평점 들 은 . 나쁘 지 ᆫ 않 지만 점 짜리 는 더더욱 아니 잖아,0,19
3,지루하 지 는 않 은데 완전 막장 이 ᆷ . 돈 주 고 보 기 에 는 .,0,18
4,만 아니 었 어도 별 다섯 개 주 었 을 터 이 ᆫ데 . 왜 로 나오 어서 저 의 ...,0,28
5,음악 이 주 가 되 ᆫ 최고 의 음악 영화,1,10
...,...,...,...
49995,오랜만 에 평점 로 이 기 ᆫ 하 었 네 킹왕짱 쌈 뽕 하 ᆫ 영화 를 만나 었 습...,1,25
49996,의지 박약 들 이나 하 는 거 다 탈영 은 일단 주인공 김대희 닮 었 고 이등병 찐따,0,18
49997,그림 도 좋 고 완성도 도 높 었 지만 . 보 는 내내 불안 하 게 만들 ᆫ다,0,18
49998,절대 보 어서 는 안 되 ᆯ 영화 . 재미 도 없 고 기분 만 잡치 고 . 한 세트...,0,26


In [5]:

df.to_csv("data/korean_movie_test.txt", sep = "\t", encoding = "UTF8")

In [6]:
print(df["word_len"].max())
print(df["word_len"].mean())
print(df["word_len"].std())

108
19.115372213937366
16.417087968759585
